In [1]:
import sys
sys.path.append('../src')
sys.path.append('../../DNSGP/GPflow')

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline 

from GPflow.likelihoods import MultiClass
from GPflow.kernels import RBF, White, Linear, Matern32, Matern52
from GPflow.svgp import SVGP
from GPflow.gpr import GPR

from scipy.cluster.vq import kmeans2

from get_data import get_mnist_data
from dgp import DGP

X, Y, Xs, Ys = get_mnist_data()


Extracting /Users/hughsalimbeni/Documents/dnsgpfiles/data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/hughsalimbeni/Documents/dnsgpfiles/data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/hughsalimbeni/Documents/dnsgpfiles/data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/hughsalimbeni/Documents/dnsgpfiles/data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
M = 100
Z = kmeans2(X, M, minit='points')[0]

In [3]:
# f, axs = plt.subplots(10, 10, figsize=(6, 6))
# for z, ax in zip(Z[:10], axs.ravel()):
#     ax.matshow(z.reshape(28, 28))

In [4]:
def make_dgp(L):
    kernels = [RBF(784, lengthscales=2., variance=2.)]
    for l in range(L-1):
        kernels.append(RBF(30, lengthscales=2., variance=2.))
    model = DGP(X, Y, Z, kernels, MultiClass(10), 
                num_samples=1,
                minibatch_size=10000,
                num_latent_Y=10)

    for layer in model.layers[:-1]:
        layer.q_sqrt = layer.q_sqrt.value * 1e-5
    
    return model
m_dgp1 = make_dgp(1)
m_dgp2 = make_dgp(2)

In [5]:
print m_dgp1.compute_log_likelihood()

AttributeError: 'module' object has no attribute 'get_shape'